# Tutorial 3: Failure to converge

This is achieved by fixing just one node which makes stiffness matrix ill conditioned.

In [1]:
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [2]:
# for saving files later
tutorials_dir = Path.cwd()

In [3]:
# Set up a 2D cantilever beam under a uniform downward traction on the top edge.
# The left edge (x=0) is clamped. For small deflection, compare the tip displacement
# at x=L, y=H/2 with the Euler–Bernoulli beam formula:
# For a cantilever of length L, height H, uniform load q, plane strain,
#   Material:
#         E = 100000, nu = 0.3.
#     Derived:
#         mu    = E/(2*(1+nu))
#         kappa = E/(3*(1-2*nu))
    
#     Analytical tip deflection for a cantilever beam under uniform load q is:
#         w(L) = q*L^4 / (8*E*I),   I = H^3/12.


# --- Beam geometry ---
L = 10.0   # length in x
H = 1.0    # height in y
nx = 6    # number of elements along length
ny = 1     # number of elements along height

ele_type = "D2_nn3_tri"  # 2D, 3-node triangle (quad)
ndof = 2                  # 2 DOFs per node (x, y)

In [ ]:
# Generate a rectangular mesh
coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)
# coords: shape (n_nodes, 2)
# connect: shape (n_nodes_per_elem, n_elems)

# --- Identify boundaries ---
boundary_nodes, boundary_edges = pre.identify_rect_boundaries(
    coords, connect, ele_type, x_lower=0.0, x_upper=L, y_lower=0.0, y_upper=H
)

# Uniform downward traction on the top edge (y=H)
# Let q be negative in the y-direction
q = -1  # load per unit length in x
# For a 2D plane strain problem, this is a traction (tx, ty) = (0, q)
dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, q)

In [ ]:
# Combine boundary conditions 
fixed_nodes = np.array([
    [0],      # DOF 0 (x-direction of node 0)
    [1.0],    # Load stepping multiplier
    [0.0]     # Prescribed value
])

In [6]:
# --- Material properties ---
E = 100000.0
nu = 0.3
# mu = E / (2.0 * (1.0 + nu))
# kappa = E / (3.0 * (1.0 - 2.0 * nu))
mu = E / (2.0 * (1.0 + nu))
#kappa = E / (2.0 * (1.0 - nu))
kappa = E / (3.0 * (1.0 - 2.0 * nu))

material_props = np.array([mu, kappa])
print(f"Material properties: mu={mu:.3f}, kappa={kappa:.3f}")

Material properties: mu=38461.538, kappa=83333.333


In [7]:
# Number of incremental load steps
nr_num_steps = 1

# --- Solve with your hyperelastic solver ---
displacements_all, nr_info_all = hyperelastic_solver(
    material_props,
    ele_type,
    coords.T,      # shape (2, n_nodes)
    connect.T,     # shape (n_nodes_per_elem, n_elems)
    fixed_nodes,
    dload_info,
    nr_print=True,
    nr_num_steps=nr_num_steps,
    nr_tol=1e-10,
    nr_maxit=30,
)

final_disp = displacements_all[-1]  # shape: (n_nodes*ndof,)


Step 0, load factor = 1.000
Iteration 1, Correction=1.000000e+00, Residual=1.395957e-01, tolerance=1.000000e-10
Iteration 2, Correction=1.084881e+00, Residual=1.973234e+34, tolerance=1.000000e-10
Iteration 3, Correction=6.501263e-01, Residual=6.413592e+33, tolerance=1.000000e-10
Iteration 4, Correction=2.032814e-01, Residual=1.900324e+33, tolerance=1.000000e-10
Iteration 5, Correction=1.884458e-01, Residual=5.630588e+32, tolerance=1.000000e-10
Iteration 6, Correction=1.504421e-01, Residual=1.668322e+32, tolerance=1.000000e-10
Iteration 7, Correction=3.849221e-02, Residual=4.943178e+31, tolerance=1.000000e-10
Iteration 8, Correction=2.710188e-02, Residual=1.464645e+31, tolerance=1.000000e-10
Iteration 9, Correction=2.247448e-02, Residual=4.339689e+30, tolerance=1.000000e-10
Iteration 10, Correction=2.318058e-02, Residual=1.285834e+30, tolerance=1.000000e-10
Iteration 11, Correction=1.520661e-02, Residual=3.809878e+29, tolerance=1.000000e-10
Iteration 12, Correction=8.824632e-03, Residua

User can see that the Residual has become very large and the soler failed to converge within the provided maximum number of iterations.

In [8]:
# --- Compute the tip displacement from the FEA result ---
# We'll pick a node near x=L, y=H/2
tip_node = None
tol = 1e-3
for i, (x, y) in enumerate(coords):
    if abs(x - L) < tol and abs(y - H/2) < H/(2*ny):
        tip_node = i
        break
if tip_node is None:
    raise ValueError("Could not find tip node near x=L, y=H/2.")

tip_disp_y = final_disp[ndof*tip_node + 1]  # the y-component of displacement

ValueError: Could not find tip node near x=L, y=H/2.

Here we can see the effect of making a poor mesh as post processing also becomes difficult as the point of interest is far away from the any point on mesh.